In [1]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value

class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:   #不管是什麼錯誤
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()
# model_dump 將對象轉化為Dict，之後就可以調用python標準庫序列化json字串，会序列化嵌套对象
# 也可以使用dict(model)将对象转化为字典，但嵌套对象不会被转化为字典。
df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,2,26,2024-07-02 23:14:21
1,大安區,21,2,19,2024-07-02 23:03:20
2,大安區,16,16,0,2024-07-02 22:48:15
3,大安區,11,6,5,2024-07-02 23:14:21
4,大安區,16,3,13,2024-07-02 23:12:19
...,...,...,...,...,...
1425,臺大公館校區,30,0,25,2024-07-02 21:19:14
1426,臺大公館校區,20,3,17,2024-07-02 22:44:19
1427,臺大公館校區,24,14,9,2024-07-02 23:10:28
1428,臺大公館校區,40,1,39,2024-07-02 23:14:20


groupby實體.統計function()  
若要使用多個function：.agg(   ex:count(),sum()  )

In [2]:
# 使用by  ，會自動將by的欄位 變成索引
# [[選取我所要統計的columns]]
df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].sum()

,總數量,可借數量,可還數量
行政區域,,,
中山區,3819,1438,2332
中正區,3209,1263,1891
信義區,3331,945,2274
內湖區,3499,1322,2131
北投區,2472,906,1526
南港區,2317,1091,1183
士林區,2962,1275,1605
大同區,1630,718,899
大安區,4987,1946,2946


In [3]:
df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg(['sum','count'])

總數量        可借數量        可還數量      
         sum count   sum count   sum count
行政區域                                      
中山區     3819   156  1438   156  2332   156
中正區     3209   121  1263   121  1891   121
信義區     3331   106   945   106  2274   106
內湖區     3499   168  1322   168  2131   168
北投區     2472    94   906    94  1526    94
南港區     2317    92  1091    92  1183    92
士林區     2962   128  1275   128  1605   128
大同區     1630    62   718    62   899    62
大安區     4987   185  1946   185  2946   185
文山區     2203    97  1043    97  1153    97
松山區     2343    88   799    88  1518    88
臺大公館校區  1437    53   366    53  1057    53
萬華區     2057    80  1111    80   891    80

In [4]:
# 多個統計function，使用 agg
df1 = df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg([('車輛數','sum'),('站點數','count')])
df1

總數量       可借數量       可還數量     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
行政區域                                   
中山區     3819  156  1438  156  2332  156
中正區     3209  121  1263  121  1891  121
信義區     3331  106   945  106  2274  106
內湖區     3499  168  1322  168  2131  168
北投區     2472   94   906   94  1526   94
南港區     2317   92  1091   92  1183   92
士林區     2962  128  1275  128  1605  128
大同區     1630   62   718   62   899   62
大安區     4987  185  1946  185  2946  185
文山區     2203   97  1043   97  1153   97
松山區     2343   88   799   88  1518   88
臺大公館校區  1437   53   366   53  1057   53
萬華區     2057   80  1111   80   891   80

#### stacking 和 unstacking 被廣泛用於改變正在考慮的 DataFrame 的形狀。 使行變成列，列相應地變成行。
DataFrame.stack(level=-1, dropna=_NoDefault.no_default, sort=_NoDefault.no_default, future_stack=False)
- Stack the prescribed level(s) from columns to index.

In [5]:
#pivot 樞紐分析
#stack,unstack
#欄位有名稱
s1 = df1.stack(level=[0,1])
s1

C:\Users\Han PC\AppData\Local\Temp\ipykernel_2544\2724416305.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  s1 = df1.stack(level=[0,1])


行政區域           
中山區   可借數量  車輛數    1438
            站點數     156
      可還數量  車輛數    2332
            站點數     156
      總數量   車輛數    3819
                   ... 
萬華區   可借數量  站點數      80
      可還數量  車輛數     891
            站點數      80
      總數量   車輛數    2057
            站點數      80
Length: 78, dtype: int64

In [6]:
s1.index.names = ['行政區域','主題','數量']
s1

行政區域  主題    數量 
中山區   可借數量  車輛數    1438
            站點數     156
      可還數量  車輛數    2332
            站點數     156
      總數量   車輛數    3819
                   ... 
萬華區   可借數量  站點數      80
      可還數量  車輛數     891
            站點數      80
      總數量   車輛數    2057
            站點數      80
Length: 78, dtype: int64

In [9]:
s1.unstack(level=['數量'])

數量            車輛數  站點數
行政區域   主題             
中山區    可借數量  1438  156
       可還數量  2332  156
       總數量   3819  156
中正區    可借數量  1263  121
       可還數量  1891  121
       總數量   3209  121
信義區    可借數量   945  106
       可還數量  2274  106
       總數量   3331  106
內湖區    可借數量  1322  168
       可還數量  2131  168
       總數量   3499  168
北投區    可借數量   906   94
       可還數量  1526   94
       總數量   2472   94
南港區    可借數量  1091   92
       可還數量  1183   92
       總數量   2317   92
士林區    可借數量  1275  128
       可還數量  1605  128
       總數量   2962  128
大同區    可借數量   718   62
       可還數量   899   62
       總數量   1630   62
大安區    可借數量  1946  185
       可還數量  2946  185
       總數量   4987  185
文山區    可借數量  1043   97
       可還數量  1153   97
       總數量   2203   97
松山區    可借數量   799   88
       可還數量  1518   88
       總數量   2343   88
臺大公館校區 可借數量   366   53
       可還數量  1057   53
       總數量   1437   53
萬華區    可借數量  1111   80
       可還數量   891   80
       總數量   2057   80

In [10]:
s1.unstack(level=['主題'])

主題          可借數量  可還數量   總數量
行政區域   數量                   
中山區    車輛數  1438  2332  3819
       站點數   156   156   156
中正區    車輛數  1263  1891  3209
       站點數   121   121   121
信義區    車輛數   945  2274  3331
       站點數   106   106   106
內湖區    車輛數  1322  2131  3499
       站點數   168   168   168
北投區    車輛數   906  1526  2472
       站點數    94    94    94
南港區    車輛數  1091  1183  2317
       站點數    92    92    92
士林區    車輛數  1275  1605  2962
       站點數   128   128   128
大同區    車輛數   718   899  1630
       站點數    62    62    62
大安區    車輛數  1946  2946  4987
       站點數   185   185   185
文山區    車輛數  1043  1153  2203
       站點數    97    97    97
松山區    車輛數   799  1518  2343
       站點數    88    88    88
臺大公館校區 車輛數   366  1057  1437
       站點數    53    53    53
萬華區    車輛數  1111   891  2057
       站點數    80    80    80